# COMP0124: Multi-Agent Artificial Intelligence

# Group project: Real-time bidding auctions

**Group #7: Oliviero Balbinetti, Mauricio Caballero, Paul Melkert**

Importing libraries.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from glob import glob
from scipy import stats

Importing data in pandas DataFrames.

In [2]:
#Defining directory.
Path = '/Users/olivierobalbinetti/Desktop/University College London/Term 2'\
       '/Multi Agents Artificial Intelligence/Courseworks/Group coursework/'\
       'Data'

#Importing data.
Datasets = {}
os.chdir(Path)

for Filename in glob('*.csv'):
    Datasets[Filename[:-4].title()] = pd.read_csv(Filename, sep = ',')
    
#Feature engineering.
Total_data = sum([len(Datasets[key]) for key in Datasets.keys()])

High level study of DataFrame.

In [3]:
#Amount of data.
print('High level study:\n')
print('Data exploration - datasets structure:')

for key in Datasets.keys():
    print('\n• ' + key + ' dataset:')
    
    dataframe = Datasets[key]
    print(' - Amount of data: %d.' %len(dataframe))
    print(' - Percentage of data (total): %.2f%%.'
          %(len(dataframe)/Total_data*100))
    print(' - Number of columns: %d.' %len(dataframe.columns))
    print(' - Column names:', *dataframe.columns, sep=' ')
    
#Data description.
print('\nData exploration - columns description:\n')

print('• click: [0,1] variable describing feebacks (1 if clicked).')
print('• weekday: [0,1,..,6] variable describing week days.')
print('• hour: [0,1,..,23] variable describing day hours.')
print('• bidid: alphanumeric string identifying each single event.')
print('• userid: alphanumeric string identifying each single user.')
print('• useragent: string containing browser user agent used (splitted).')
print('• IP: string containing first three bytes of user IP address.')
print('• region: province or state numeric ID where the user visits from.')
print('• city: city numeric ID where the user visits from.')
print('• adexchange: [nan,1,2,3,4] variable identifying ad exchange.')
print('• domain: alphanumeric string identifying hosting webpage domain.')
print('• url: alphanumeric string identifying URL (avail. to DPS).')
print('• urlid: alphanumeric string identifying URL (prov. by ad exchange).')
print('• slotid: unique ID identifying ad impression location on web page.')
print('• slotwidth: variable representing the width of the slot.')
print('• slotheight: variable representing the height of the slot.')
print('• slotvisibility: string identifying slot position wrt the fold.')
print('• slotformat: variable representing slot types.')
print('• slotprice: variable representing slot reserve price.')
print('• creative: alphanumeric string identifying advertiser ad creative.')
print('• bidprice: variable describing price DSP bids.')
print('• payprice: variable describing price DSP pays on winning.')
print('• keypage: alphanumeric string identifying main page URL.')
print('• advertiser: unique ID identifying advertisers.')
print('• usertag: variable containing list of tags describing each user.')

#Advertiser description.
print('\nData exploration - advertisers description:\n')

print('• [Advertiser ID: 1458]: Industry: Chinese vertical e-commerce.')
print('• [Advertiser ID: 2259]: Industry: Milk powder.')
print('• [Advertiser ID: 2261]: Industry: Telecom.')
print('• [Advertiser ID: 2821]: Industry: Footwear.')
print('• [Advertiser ID: 2997]: Industry: Mobile e-commerce app install.')
print('• [Advertiser ID: 3358]: Industry: Software.')
print('• [Advertiser ID: 3386]: Industry: International e-commerce.')
print('• [Advertiser ID: 3427]: Industry: Oil.')
print('• [Advertiser ID: 3476]: Industry: Tire.')

High level study:

Data exploration - datasets structure:

• Validation dataset:
 - Amount of data: 303925.
 - Percentage of data (total): 10.00%.
 - Number of columns: 25.
 - Column names: click weekday hour bidid userid useragent IP region city adexchange domain url urlid slotid slotwidth slotheight slotvisibility slotformat slotprice creative bidprice payprice keypage advertiser usertag

• Test dataset:
 - Amount of data: 303375.
 - Percentage of data (total): 9.99%.
 - Number of columns: 22.
 - Column names: weekday hour bidid userid useragent IP region city adexchange domain url urlid slotid slotwidth slotheight slotvisibility slotformat slotprice creative keypage advertiser usertag

• Train dataset:
 - Amount of data: 2430981.
 - Percentage of data (total): 80.01%.
 - Number of columns: 25.
 - Column names: click weekday hour bidid userid useragent IP region city adexchange domain url urlid slotid slotwidth slotheight slotvisibility slotformat slotprice creative bidprice payprice

Descriptive analytics.

In [4]:
#Advertiser statistics.
Advs = dict(enumerate(sorted(set(Datasets['Train']['advertiser'].values))))
Advs = {value:key for key,value in Advs.items()}

Train_statistics = []
Matrix_train = np.zeros((len(Advs),6))

Validation_statistics = []
Matrix_validation = np.zeros((len(Advs),6))

#Loping on advertisers.
for adv in Advs.keys():
    
    #Train set analysis.
    data = Datasets['Train'][Datasets['Train']['advertiser'] == adv]
    Matrix_train[Advs[adv],0] = len(data)
    Matrix_train[Advs[adv],1] = sum(data['click'])
    Matrix_train[Advs[adv],2] = sum(data['payprice'])/1000
    Matrix_train[Advs[adv],3] = sum(data['click'])/len(data)*100
    Matrix_train[Advs[adv],4] = np.mean(data['payprice'])/1000
    Matrix_train[Advs[adv],5] = sum(data['payprice'])/(sum(data['click'])*1000)
    
    #Validation set analysis.
    data = Datasets['Validation'][Datasets['Validation']['advertiser'] == adv]
    Matrix_validation[Advs[adv],0] = len(data)
    Matrix_validation[Advs[adv],1] = sum(data['click'])
    Matrix_validation[Advs[adv],2] = sum(data['payprice'])/1000
    Matrix_validation[Advs[adv],3] = sum(data['click'])/len(data)*100
    Matrix_validation[Advs[adv],4] = np.mean(data['payprice'])/1000
    Matrix_validation[Advs[adv],5] = sum(data['payprice'])/(sum(data['click'])*1000)
    
#Overall statistics.
Train_statistics.append(np.sum(Matrix_train, axis=0))
Train_statistics.append(np.mean(Matrix_train, axis=0))
Train_statistics.append(stats.sem(Matrix_train, axis=0))
Train_statistics.append(np.std(Matrix_train, axis=0))
Train_statistics.append(stats.skew(Matrix_train, axis=0))
Train_statistics.append(stats.kurtosis(Matrix_train, axis=0, fisher=True))

Validation_statistics.append(np.sum(Matrix_validation, axis=0))
Validation_statistics.append(np.mean(Matrix_validation, axis=0))
Validation_statistics.append(stats.sem(Matrix_validation, axis=0))
Validation_statistics.append(np.std(Matrix_validation, axis=0))
Validation_statistics.append(stats.skew(Matrix_validation, axis=0))
Validation_statistics.append(stats.kurtosis(Matrix_validation, axis=0, fisher=True))

#Printing results.
Df_train_advs = pd.DataFrame(Matrix_train, index=Advs.keys(),
                             columns=['Imps', 'Click', 'Cost', 'CTR (%)',
                                      'avg CPM', 'eCPC'])
Df_validation_advs = pd.DataFrame(Matrix_validation, index=Advs.keys(),
                                  columns=['Imps', 'Click', 'Cost', 'CTR (%)',
                                           'avg CPM', 'eCPC'])

Df_train_stats = pd.DataFrame(np.array(Train_statistics),
                              index=['Total', 'Mean', 'Std err','Std',
                                     'Skew', 'Kurt'],
                              columns=['Imps', 'Click', 'Cost', 'CTR',
                                       'avg CPM', 'eCPC'])

Df_validation_stats = pd.DataFrame(np.array(Validation_statistics),
                                   index=['Total', 'Mean', 'Std err','Std',
                                          'Skew', 'Kurt'],
                                   columns=['Imps', 'Click', 'Cost', 'CTR',
                                            'avg CPM', 'eCPC'])

print('Descriptive analytics:\n')
print('Total number of advertisers in datasets: %d.\n' %len(Advs))

print('• Train set:\n')
print(' - Advertisers statistics:')
print(Df_train_advs)
print('\n - Summarization statistics:')
print(Df_train_stats)

print('\n• Validation set:\n')
print(' - Advertisers statistics:')
print(Df_validation_advs)
print('\n - Summarization statistics:')
print(Df_validation_stats)

Descriptive analytics:

Total number of advertisers in datasets: 9.

• Train set:

 - Advertisers statistics:
          Imps  Click       Cost   CTR (%)   avg CPM        eCPC
1458  492353.0  385.0  33968.736  0.078196  0.068993   88.230483
2259  133673.0   43.0  12428.238  0.032168  0.092975  289.028791
2261  110122.0   36.0   9873.779  0.032691  0.089662  274.271639
2821  211366.0  131.0  18828.044  0.061978  0.089078  143.725527
2997   49829.0  217.0   3129.267  0.435489  0.062800   14.420585
3358  264956.0  202.0  22447.231  0.076239  0.084721  111.124906
3386  455041.0  320.0  34931.823  0.070323  0.076766  109.161947
3427  402806.0  272.0  30458.711  0.067526  0.075616  111.980555
3476  310835.0  187.0  23918.779  0.060161  0.076950  127.907909

 - Summarization statistics:
                 Imps        Click           Cost       CTR   avg CPM  \
Total    2.430981e+06  1793.000000  189984.608000  0.914771  0.717561   
Mean     2.701090e+05   199.222222   21109.400889  0.101641  0.0

In [6]:
a = Datasets['Train']

In [7]:
a

,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,...,slotheight,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag
0,0,5,22,b7bea80521fdecd95d2d761a38c91c3f09618066,2e880fb7d690cf7377b2e42e701728e3f3c0e4c1,windows_ie,125.37.175.*,2,2,2.0,...,200,2,0,5,a4f763f78ef3eedfe614263b94a8924e,238,5,0f951a030abdaedd733ee8d114ce2944,3427,NaN
1,0,1,20,4f51205475678f5a124bc76b2c54163bf8eaa7eb,3a1fe01360ff8100e7d006b83b77a3e4c01d928c,windows_chrome,171.36.92.*,238,239,1.0,...,250,FourthView,Na,0,10722,294,23,NaN,2821,NaN
2,0,3,13,b604e3fd054a658ab7ced4285ebf2ef54d2bd890,801d18a056b6fe6b06a794aef17fb0d6daff2414,windows_ie,59.46.106.*,40,41,2.0,...,250,2,0,5,798b2d49952d77f1eace9f23c210d0b5,238,24,0f951a030abdaedd733ee8d114ce2944,3427,"10052,10006,13866,10110"
3,0,6,23,0348beeae93e561584c3b50fc9e7746a33048ad7,0d6eaf2259699990e38a1fc5116f112070b9ecdc,windows_ie,114.250.226.*,1,1,1.0,...,600,2,1,0,cb7c76e7784031272e37af8e7e9b062c,300,25,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10063,10111"
4,0,5,6,268149c1789bce2bc9798ffd97ec431219bafeb3,a239d9bb642460d974ba67f85e63b8d3e214da0e,windows_ie,183.63.192.*,216,233,2.0,...,90,OtherView,Na,133,7330,277,133,NaN,2259,NaN
5,0,4,17,1be2cf4a47a2a6aee4b0fa64d1b786d3897be4f0,8810fd690f88ddfb797867e3e08e7816f274bd46,windows_ie,119.45.182.*,276,277,1.0,...,280,2,1,0,77819d3e0b3467fe5c7b16d68ad923a1,300,123,bebefa5efe83beee17a3d245e7c5085b,1458,"10006,10077,10063,10057,10110"
6,0,3,12,6b4f23dd215a5f11136c2f29214236cfee1f0122,fbd1bcaf2b33108d2c7f88ac41f0b0194b508a9b,windows_ie,112.82.93.*,80,89,3.0,...,90,0,0,20,86c2543527c86a893d4d4f68810a0416,300,20,43f4a37f42a7c5e6219e2601b26c6976,3386,"10083,10024,10063,13496,10093,10006,10057,10110"
7,0,3,16,f7c1d8ea177211249456c79e194617ce107bc077,9b539ba886146562a3991fd9f94bb9f9bad1f647,windows_ie,118.75.92.*,15,19,1.0,...,600,2,1,0,cb7c76e7784031272e37af8e7e9b062c,300,134,bebefa5efe83beee17a3d245e7c5085b,1458,"10063,10024,13800,13866,10006,10111"
8,0,5,14,9e97694096fe8692851048a893231a7f7ff87922,ddc9c85b6a5726aafaa4c42239e90fb7a44a88d7,windows_ie,61.185.128.*,333,342,3.0,...,90,0,0,70,832b91d59d0cb5731431653204a76c0e,300,70,bebefa5efe83beee17a3d245e7c5085b,1458,"10063,10006,10083,10110"
9,0,5,0,d1b7c7fdb88ce3aa78a540bb29848dd77329a574,63b4f9a3638f81af361ab3c4190fd3e1b5e70fea,windows_chrome,117.9.211.*,2,2,3.0,...,90,0,0,70,a10c31a8ff5f42930b4c34035e523886,241,77,d29e59bf0f7f8243858b8183f14d4412,3358,"13403,10031,10063,10083,10006,10077,10075,10110"


In [16]:
new = a['useragent'].str.split('_', n=1, expand=True)

In [19]:
new

,0,1
0,windows,ie
1,windows,chrome
2,windows,ie
3,windows,ie
4,windows,ie
5,windows,ie
6,windows,ie
7,windows,ie
8,windows,ie
9,windows,chrome


In [17]:
a['OS'] = new[0]
a['Browser'] = new[1]

In [18]:
a

,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,...,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag,OS,Browser
0,0,5,22,b7bea80521fdecd95d2d761a38c91c3f09618066,2e880fb7d690cf7377b2e42e701728e3f3c0e4c1,windows_ie,125.37.175.*,2,2,2.0,...,0,5,a4f763f78ef3eedfe614263b94a8924e,238,5,0f951a030abdaedd733ee8d114ce2944,3427,NaN,windows,ie
1,0,1,20,4f51205475678f5a124bc76b2c54163bf8eaa7eb,3a1fe01360ff8100e7d006b83b77a3e4c01d928c,windows_chrome,171.36.92.*,238,239,1.0,...,Na,0,10722,294,23,NaN,2821,NaN,windows,chrome
2,0,3,13,b604e3fd054a658ab7ced4285ebf2ef54d2bd890,801d18a056b6fe6b06a794aef17fb0d6daff2414,windows_ie,59.46.106.*,40,41,2.0,...,0,5,798b2d49952d77f1eace9f23c210d0b5,238,24,0f951a030abdaedd733ee8d114ce2944,3427,"10052,10006,13866,10110",windows,ie
3,0,6,23,0348beeae93e561584c3b50fc9e7746a33048ad7,0d6eaf2259699990e38a1fc5116f112070b9ecdc,windows_ie,114.250.226.*,1,1,1.0,...,1,0,cb7c76e7784031272e37af8e7e9b062c,300,25,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10063,10111",windows,ie
4,0,5,6,268149c1789bce2bc9798ffd97ec431219bafeb3,a239d9bb642460d974ba67f85e63b8d3e214da0e,windows_ie,183.63.192.*,216,233,2.0,...,Na,133,7330,277,133,NaN,2259,NaN,windows,ie
5,0,4,17,1be2cf4a47a2a6aee4b0fa64d1b786d3897be4f0,8810fd690f88ddfb797867e3e08e7816f274bd46,windows_ie,119.45.182.*,276,277,1.0,...,1,0,77819d3e0b3467fe5c7b16d68ad923a1,300,123,bebefa5efe83beee17a3d245e7c5085b,1458,"10006,10077,10063,10057,10110",windows,ie
6,0,3,12,6b4f23dd215a5f11136c2f29214236cfee1f0122,fbd1bcaf2b33108d2c7f88ac41f0b0194b508a9b,windows_ie,112.82.93.*,80,89,3.0,...,0,20,86c2543527c86a893d4d4f68810a0416,300,20,43f4a37f42a7c5e6219e2601b26c6976,3386,"10083,10024,10063,13496,10093,10006,10057,10110",windows,ie
7,0,3,16,f7c1d8ea177211249456c79e194617ce107bc077,9b539ba886146562a3991fd9f94bb9f9bad1f647,windows_ie,118.75.92.*,15,19,1.0,...,1,0,cb7c76e7784031272e37af8e7e9b062c,300,134,bebefa5efe83beee17a3d245e7c5085b,1458,"10063,10024,13800,13866,10006,10111",windows,ie
8,0,5,14,9e97694096fe8692851048a893231a7f7ff87922,ddc9c85b6a5726aafaa4c42239e90fb7a44a88d7,windows_ie,61.185.128.*,333,342,3.0,...,0,70,832b91d59d0cb5731431653204a76c0e,300,70,bebefa5efe83beee17a3d245e7c5085b,1458,"10063,10006,10083,10110",windows,ie
9,0,5,0,d1b7c7fdb88ce3aa78a540bb29848dd77329a574,63b4f9a3638f81af361ab3c4190fd3e1b5e70fea,windows_chrome,117.9.211.*,2,2,3.0,...,0,70,a10c31a8ff5f42930b4c34035e523886,241,77,d29e59bf0f7f8243858b8183f14d4412,3358,"13403,10031,10063,10083,10006,10077,10075,10110",windows,chrome
